In [1]:
%pip install flask transformers torch 

Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install sentencepiece --quiet

In [3]:
!python -m pip install --upgrade pip

In [4]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Load tokenizer
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-large")

# Load model in TensorFlow
model_pt = T5ForConditionalGeneration.from_pretrained("google/flan-t5-large")

# Example: encode some text and get a prediction
input_text = "Translate English to French: How are you?"
inputs = tokenizer(input_text, return_tensors="pt")
outputs = model_pt.generate(**inputs)
translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Translated text (PyTorch):", translated_text)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Translated text (PyTorch): Comment êtes-vous?


In [14]:
%pip install flask-cors

Note: you may need to restart the kernel to use updated packages.


In [19]:
from flask import Flask, request, jsonify
from transformers import pipeline
import torch
from flask_cors import CORS, cross_origin
import threading
import time

# Check for GPU availability and set device
device = 0 if torch.cuda.is_available() else -1

# Load the translation model
translator = pipeline("text2text-generation", model="google/flan-t5-large", device=device)

# Initialize Flask
app = Flask(__name__)
CORS(app)

@app.after_request
def after_request(response):
    response.headers.add('Access-Control-Allow-Origin', '*')
    response.headers.add('Access-Control-Allow-Headers', 'Content-Type')
    response.headers.add('Access-Control-Allow-Methods', 'POST, OPTIONS')
    return response

@app.route('/')
def home():
    return "🟢 Translation API is running!"

@app.route('/translate', methods=['POST', 'OPTIONS'])  # IMPORTANT: Allow POST and OPTIONS here
def translate_text():
    if request.method == "OPTIONS":
        return '', 204  # Respond to preflight
    try:
        # Get request data
        data = request.get_json()

        # Validate input
        text = data.get('text', '').strip()
        if not text:
            return jsonify({'error': 'Text must be a non-empty string'}), 400

        source_lang = data.get('source_lang', 'English').strip()
        target_lang = data.get('target_lang', 'French').strip()

        if not source_lang or not target_lang:
            return jsonify({'error': 'Both source_lang and target_lang must be provided'}), 400

        # Create dynamic prompt
        prompt = f"translate {source_lang} to {target_lang}: {text}"

        # Translate
        translated_text = translator(prompt, max_length=256, do_sample=False)

        return jsonify({
            'translation': translated_text[0]['generated_text'],
            'source_lang': source_lang,
            'target_lang': target_lang
        })

    except Exception as e:
        return jsonify({'error': str(e)}), 500

def run_flask():
    app.run(host='0.0.0.0', port=5000, debug=False, use_reloader=False)

# Run Flask app in a separate thread
flask_thread = threading.Thread(target=run_flask)
flask_thread.start()

time.sleep(2)  # Allow server time to start


Device set to use cpu


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://169.254.95.18:5000
Press CTRL+C to quit
